#### 数据预处理
提取五个信息矩阵
- 提取A访问矩阵，
- 提取A的访问区间，
- 提取B访问矩阵，
- 提取A的带索引访问矩阵，
- 提取B的带索引访问矩阵，

In [2]:
import pandas as pd
import numpy as np
import re

acceess_file_A        = './data/TEST1_A_access.txt'  
acceess_file_B        = './data/TEST1_B_access.txt'  
acceess_interval_file = './data/TEST1_A_access_interval.txt'  

######################## 提取A访问矩阵
with open(acceess_file_A, 'r') as file:
    lines = file.readlines()

# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, y, z) 中的z提取
def extract_func_1(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        zs = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', last_match)
        return [int(t[2]) for t in zs]
    return []

# 提取每行最后一个方括号内的内容
access_matrix_a = []
for line in lines:
    extracted_data = extract_func_1(line)
    access_matrix_a.append(extracted_data)

# 将提取的数据转换为 NumPy 数组，每行 16 个元素，不足的补 -1
max_length = 64
access_matrix_np_a = np.array([data + [-1] * (max_length - len(data)) if len(data) < max_length else data[:max_length] for data in access_matrix_a])
# access_matrix
# print(access_matrix_np.tolist())


######################## 提取ss_interval #######################
with open(acceess_interval_file, 'r') as file:
    lines = file.readlines()

# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (start, end) 中的start或end提取
def extract_interval(text, idx):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        intervals = re.findall(r'\((\d+),\s*(\d+)\)', last_match)
        # print(intervals)
        return [int(t[idx]) for t in intervals]
    return []


start_matrix = []
end_matrix = []
for line in lines:
    start = extract_interval(line, 0)
    end = extract_interval(line, 1)
    start_matrix.append(start)
    end_matrix.append(end)

######################## 提取B访问矩阵
with open(acceess_file_B, 'r') as file:
    lines = file.readlines()

# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, z) 中的z提取
def extract_func_2(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        zs = re.findall(r'\((\d+),\s*(\d+)\)', last_match)
        return [int(t[1]) for t in zs]
    return []

# 提取每行最后一个方括号内的内容
access_matrix_b = []
for line in lines:
    extracted_data = extract_func_2(line)
    access_matrix_b.append(extracted_data)

# 将提取的数据转换为 NumPy 数组，每行 64 个元素，不足的补 -1
max_length = 64 # 脑子抽了，下面-1记得扔
access_matrix_np_b = np.array([data + [-1] * (max_length - len(data)) if len(data) < max_length else data[:max_length] for data in access_matrix_b])


######################## 提取A的索引访问矩阵
with open(acceess_file_A, 'r') as file:
    lines = file.readlines()
    
# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, y, z) 提取
def extract_func_3(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        tuples = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', last_match)
        return [tuple(map(int, t)) for t in tuples]
    return []

access_idx_matrix_a = []
for idx, line in enumerate(lines):
    extracted_data = extract_func_3(line)
    access_idx_matrix_a.append((idx, extracted_data))


######################## 提取B的索引访问矩阵
with open(acceess_file_B, 'r') as file:
    lines = file.readlines()
    
# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, z) 提取
def extract_func_4(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        tuples = re.findall(r'\((\d+),\s*(\d+)\)', last_match)
        return [tuple(map(int, t)) for t in tuples]
    return []

access_idx_matrix_b = []
for idx, line in enumerate(lines):
    extracted_data = extract_func_4(line)
    access_idx_matrix_b.append((idx, extracted_data))



In [3]:
!pwd
!rm ./output/*.txt

/home/shiroha/Code/Backend/SpMM-simulator/spmmsim/model/hw_model/playground


In [4]:
import sys
sys.path.append("../")

from system_model.prefetcher.SP.stride_prefetch import StridePrefetcherGroup
from system_model.prefetcher.IMP.imp import IMPGroup
from system_model.prefetcher.DVR.DVR import DVR
from system_model.prefetcher.NVR.NVR import NVR
# from memory_model.memsim import MemSim

# 这个函数应该究极耗时
# def A_idx_B(a_num, A_matrix, B_matrix, num_has_get):
#     # num_has_get=(i, num_has_get)
#     # 是指这一轮之前已经在i这里取了几个数了，这次应该在第num_has_get+1个地方取
#     for a_idx, data in A_matrix:
#         for row, col, a_addr in data:
#             if a_num == a_addr:
#                 b_idx, data = B_matrix[a_idx]
#                 b_addr_matched_set = [b_addr for i, b_addr in data if i == row]
#                 # for i, b_addr in data:
#                 #     if i == row:
#                 num_to_get = num_has_get[i][1]+1
#                 return b_addr_matched_set[num_to_get], 
#     return -1

from collections import defaultdict

def A_idx_B(a_num, A_matrix, B_matrix, record_buffer=None):
    # 使用 defaultdict 来存储已经取过的值
    if record_buffer is None:
        record_buffer = defaultdict(list)
    
    # 遍历 A_matrix 查找 a_num 对应的地址
    for a_idx, data in A_matrix:
        for row, col, a_addr in data:
            if a_num == a_addr:
                # # 获取对应 B_matrix 的数据
                # b_idx, b_data = B_matrix[a_idx]
                # # 找到匹配的行号的所有 b_addr
                # b_addr_matched_set = [b_addr for i, b_addr in b_data if i == col]
                
                # # 检查在缓冲区中已经记录的索引
                # already_taken = len(record_buffer[(a_idx, col)])
                # if already_taken < len(b_addr_matched_set):
                #     # 记录取过的值
                #     record_buffer[(a_idx, col)].append(b_addr_matched_set[already_taken])
                #     return b_addr_matched_set[already_taken]
                # else:
                    # return -1
                    return col
    return -1

def clear_buffer(record_buffer):
    # 清空记录缓冲区
    record_buffer.clear()
    

if __name__ == "__main__":
    sp = StridePrefetcherGroup(num_prefetchers=64, default_stride=4)
    # imp = IMPGroup(num_units=16)
    imp = DVR(lbd=False, vector_size=16, ewma_alpha=0.3, initial_bound=2)
    dvr = DVR(lbd=True, vector_size=16, ewma_alpha=0.3, initial_bound=2)
    nvr = NVR()
    

    mvin_instructions = access_matrix_np_a
    ss_start = start_matrix
    ss_end = end_matrix
    

    # bound = [[end - start for start, end in zip(start_list, end_list)] for start_list, end_list in zip(ss_start, ss_end)]
    bound = [[
            (end - start + 1) if (start != 0 and end != 0) else 0
            for start, end in zip(start_list, end_list)
        ]
        for start_list, end_list in zip(ss_start, ss_end)
    ]
    # print(bound)
    
    ## 索引矩阵
    idx_matrix_a = access_idx_matrix_a
    idx_matrix_b = access_idx_matrix_b

    
    # tiled_num = 3
    # for i in range(len(access_matrix_np_a)-1):
    for i in range(3):
        # print(mvin_instructions[i])
        sp_predict_addr  =  sp.prefetch(mvin_instructions[i]) 
        imp_predict_addr = imp.prefetch(bound[i], ss_start[i+1]) 
        dvr_predict_addr = dvr.prefetch(bound[i], ss_start[i+1]) 
        nvr_predict_addr = nvr.prefetch(ss_start[i+1], ss_end[i+1]) 
        actrual_addr     = mvin_instructions[i+1]
        

        np.set_printoptions(threshold=np.inf)
        prefetch_buffer = defaultdict(list)
        ### 步长预取器结果 ######################################
        sp_result_a = []
        for _, _, predicted_address, _ in sp_predict_addr:
            sp_result_a.append(int(predicted_address))
        with open(f"./output/sp_A_predict_addr.txt", "a") as file:
            file.write(f"{sp_result_a}\n")  
        print(f"sp预测A地址{sp_result_a}" )
        ########################################################
        
        ### IMP预取器结果 ######################################
        imp_result_a = []
        for _, predicted_address in imp_predict_addr:
            imp_result_a.append(int(predicted_address))
        with open(f"./output/imp_A_predict_addr.txt", "a") as file:
            file.write(f"{imp_result_a}\n")  
        print(f"imp预测A地址{imp_result_a}") # imp预测A并不是为了预取A, 它只是根据这个结果A[i]取B[A[i]](吗?)

        imp_result_b = []
        for a_addr in imp_result_a:
            b_addr = A_idx_B(a_addr, idx_matrix_a, idx_matrix_b, prefetch_buffer)
            imp_result_b.append(b_addr)
        clear_buffer(prefetch_buffer)
        with open(f"./output/imp_B_predict_addr.txt", "a") as file:
            file.write(f"{imp_result_b}\n")  
        print(f"imp预测B地址{imp_result_b}")
        ########################################################

        ### DVR预取结果 ######################################
        dvr_result_a = []
        for _, predicted_address in dvr_predict_addr:
            dvr_result_a.append(int(predicted_address))
        with open(f"./output/dvr_A_predict_addr.txt", "a") as file:
            file.write(f"{dvr_result_a}\n")  
        print(f"dvr预测A地址{dvr_result_a}")

        dvr_result_b = []
        for a_addr in dvr_result_a:
            b_addr = A_idx_B(a_addr, idx_matrix_a, idx_matrix_b, prefetch_buffer)
            dvr_result_b.append(b_addr)
        clear_buffer(prefetch_buffer)
        with open(f"./output/dvr_B_predict_addr.txt", "a") as file:
            file.write(f"{dvr_result_b}\n")  
        print(f"dvr预测B地址{dvr_result_b}")
        ########################################################

        ### NVR预取结果 ######################################
        nvr_result_a = []
        for _, predicted_address in nvr_predict_addr:
            nvr_result_a.append(int(predicted_address))
        with open(f"./output/nvr_A_predict_addr.txt", "a") as file:
            file.write(f"{nvr_result_a}\n")  
        print(f"nvr预测A地址{nvr_result_a}")
        
        nvr_result_b = []
        for a_addr in nvr_result_a:
            b_addr = A_idx_B(a_addr, idx_matrix_a, idx_matrix_b, prefetch_buffer)
            nvr_result_b.append(b_addr)
        clear_buffer(prefetch_buffer)
        with open(f"./output/nvr_B_predict_addr.txt", "a") as file:
            file.write(f"{nvr_result_b}\n")  
        print(f"nvr预测B地址{nvr_result_b}")
        ########################################################

        ### 实际地址 ######################################
        with open(f"./output/actrual_A_addr.txt", "a") as file:
            file.write(f"{actrual_addr.tolist()}\n")  
        print(f"实际A地址{actrual_addr.tolist()}")
        ########################################################
        np.set_printoptions(threshold=1000)

        
    mvin_instructions = access_matrix_np_b

    for i in range(3):
    # for i in range(len(access_matrix_np_b)-1):
        sp_predict_addr  =  sp.prefetch(mvin_instructions[i]) 
        actrual_addr     = mvin_instructions[i+1]
        
        np.set_printoptions(threshold=np.inf)
        ### 步长预取器结果 ######################################
        sp_result = []
        for _, _, predicted_address, _ in sp_predict_addr:
            sp_result.append(int(predicted_address))
        with open(f"./output/sp_B_predict_addr.txt", "a") as file:
            file.write(f"{sp_result}\n")  
        print(f"sp预测B地址{sp_result}" )
        ########################################################
        
        ### 实际地址 ######################################
        with open(f"./output/actrual_B_addr.txt", "a") as file:
            file.write(f"{actrual_addr.tolist()}\n")  
        print(f"实际B地址{actrual_addr.tolist()}")
        ########################################################
        np.set_printoptions(threshold=1000)



sp预测A地址[2, 4, 414, 842, 844, 1062, 1504, 1506, 1508, 1510, 1512, 1514, 1936, 1938, 2350, 2548, 2550, 2552, 2768, 3010, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
imp预测A地址[3, 4, 208, 0, 1, 532, 758, 759, 760, 761, 762, 763, 970, 971, 1176, 1277, 1278, 1279, 1385, 1506]
imp预测B地址[44, 54, 29, -1, 3, 31, 43, 70, 85, 95, 102, 119, 33, 59, 34, 51, 58, 86, 31, 46]
dvr预测A地址[3, 4, 90, 208, 209, 319, 0, 1, 532, 533, 644, 758, 759, 760, 0, 970, 971, 0, 1176, 1177, 1277, 1278, 1385, 1386, 1506, 1507, 1608]
dvr预测B地址[44, 54, 51, 29, 42, 31, -1, 3, 31, 121, 26, 43, 70, 85, -1, 33, 59, -1, 34, 46, 51, 58, 31, 36, 46, 47, 30]
nvr预测A地址[3, 90, 208, 209, 210, 319, 532, 644, 645, 646, 758, 970, 1176, 1177, 1277, 1385, 1386, 1387, 1506, 1507, 1608, 1609]
nvr预测B地址[44, 51, 29, 42, 49, 31, 31, 26, 40, 50, 43, 33, 34, 46, 51, 31, 36, 49, 46, 47, 30, 31]
实际A地址[3, 90, 208, 209, 210, 